In [1]:
import pandas as pd 
import psycopg2

1. Create a function for create database

In [2]:
def create_database():
    # connect to default database
    conn = psycopg2.connect('host=localhost dbname=postgres user=postgres password=root')
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # create sparktify/ command for database
    cur.execute('DROP DATABASE IF EXISTS ecommerce_sales')
    cur.execute('CREATE DATABASE ecommerce_sales')

    # close connection
    conn.close()

    # connect sparktify/ command to database
    conn = psycopg2.connect('host=localhost dbname=ecommerce_sales user=postgres password=root')
    cur = conn.cursor()

    return cur, conn

2. Create a function for drop table

In [3]:
def drop_table(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


3. Create a function for create table

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.comit()

4. Read datasets

4.1 List of order dataset

In [5]:
list_order_df = pd.read_csv(r'E-commerce/List of Orders.csv')

In [6]:
list_order_df.head()


,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


In [7]:
clean_list_order_df = list_order_df.dropna()

In [8]:
clean_list_order_df[clean_list_order_df.isnull().any(axis=1)]

,Order ID,Order Date,CustomerName,State,City


In [9]:
clean_list_order_df

,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata
...,...,...,...,...,...
495,B-26096,28-03-2019,Atharv,West Bengal,Kolkata
496,B-26097,28-03-2019,Vini,Karnataka,Bangalore
497,B-26098,29-03-2019,Pinky,Jammu and Kashmir,Kashmir
498,B-26099,30-03-2019,Bhishm,Maharashtra,Mumbai


4.2 Order details

In [10]:
order_details_df = pd.read_csv(r'E-commerce/Order Details.csv')

In [11]:
order_details_df

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones
...,...,...,...,...,...,...
1495,B-26099,835.0,267.0,5,Electronics,Phones
1496,B-26099,2366.0,552.0,5,Clothing,Trousers
1497,B-26100,828.0,230.0,2,Furniture,Chairs
1498,B-26100,34.0,10.0,2,Clothing,T-shirt


In [12]:
order_details_df.isna().sum()

Order ID        0
Amount          0
Profit          0
Quantity        0
Category        0
Sub-Category    0
dtype: int64

4.3 Sales Target

In [13]:
sales_target_df = pd.read_csv(r'E-commerce/Sales target.csv')

In [14]:
sales_target_df.head()

,Month of Order Date,Category,Target
0,Apr-18,Furniture,10400.0
1,May-18,Furniture,10500.0
2,Jun-18,Furniture,10600.0
3,Jul-18,Furniture,10800.0
4,Aug-18,Furniture,10900.0


5. Create Database

In [15]:
cur, conn = create_database()

5.1 Create clean list order table

In [16]:
cleanlistorder_table_create = ('CREATE TABLE IF NOT EXISTS list_order (\
                               "Order ID" varchar,\
                               "Order Date" varchar,\
                               CustomerName varchar,\
                               State varchar,\
                               City varchar);')

# The reason Order ID and Order Date need to put " " is because order is a function so we cant make it as column name with whitespace

In [17]:
cur.execute(cleanlistorder_table_create)
conn.commit()

5.2 Create order list table

In [18]:
orderdetails_table_create = ('CREATE TABLE IF NOT EXISTS order_details (\
                          "Order ID" varchar,\
                          Amount float,\
                          Profit float,\
                          Quantity int,\
                          Category varchar,\
                          "Sub-category" varchar);')

In [19]:
# conn.rollback()
cur.execute(orderdetails_table_create)
conn.commit()

5.3 Create Sale Target Table

In [20]:
salestarget_create_table = ('CREATE TABLE IF NOT EXISTS sales_target (\
                            "Month of Order Date" varchar,\
                            Category varchar,\
                            Target float);')

# cannot leave white space in column name

In [21]:
# conn.rollback()
cur.execute(salestarget_create_table)
conn.commit()

6. Insert Rows Into Tables

6.1 List Order

In [22]:
listorder_insert_row = ('INSERT INTO list_order ("Order ID", "Order Date", CustomerName, State, City)\
                                   VALUES (%s, %s, %s, %s, %s)')

# %s is wildcard so the data knows where to go

In [23]:
# conn.rollback()
for i, row in clean_list_order_df.iterrows():
    cur.execute(listorder_insert_row, list(row))

In [24]:
conn.commit()

6.2 Order Details

In [25]:
orderdetails_insert_row = ('INSERT INTO order_details ("Order ID", Amount, Profit, Quantity, Category, "Sub-category")\
                                   VALUES (%s, %s, %s, %s, %s, %s)')

In [28]:
# conn.rollback()

for i, row in order_details_df.iterrows():
    cur.execute(orderdetails_insert_row, list(row))

conn.commit()

6.3 Sales Target

In [31]:
salestarget_insert_row = ('INSERT INTO sales_target ("Month of Order Date", Category, Target)\
                                   VALUES (%s, %s, %s)')

In [33]:
# conn.rollback()

for i, row in sales_target_df.iterrows():
    cur.execute(salestarget_insert_row, list(row))

conn.commit()